In [ ]:
from library_dicom.dicom_processor.model.reader.Instance_RTSS import Instance_RTSS

from library_dicom.rtss_processor.model.MaskBuilder_RTSS import MaskBuilder_RTSS
from library_dicom.dicom_processor.model.Series import Series

import matplotlib.pyplot as plt
import numpy as np

### RTSTRUCT file -> Mask 

In [ ]:
rtss_path = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/RT_STRUCT/201802409 BARAT GALPIN ELIANE/103893708 Tepscan FDG/RTSTRUCT PETVCAR RTSS RTSS/RT000000.dcm'
#rtss_path = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/RT_STRUCT/rtss.dcm'
ct_path = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/RT_STRUCT/201802409 BARAT GALPIN ELIANE/103893708 Tepscan FDG/CT CT STD 2.5mm'
pet_path = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/RT_STRUCT/201802409 BARAT GALPIN ELIANE/103893708 Tepscan FDG/PT WB FDG VPHD 2min'

In [ ]:
mask_builder_rtss = MaskBuilder_RTSS(rtss_path, pet_path)

In [ ]:
from time import process_time
t1_start = process_time()
mask_4D = mask_builder_rtss.rtss_to_4D_mask()
t1_stop = process_time() 
print("diff début fin :", t1_stop-t1_start)  

In [ ]:
mask_builder_rtss.number_of_roi

In [ ]:
mask_4D.shape

In [ ]:
plt.imshow(mask_4D[:,:,237,0]) #0 pour ROI 1
plt.show()
#3Dslicer <= 309 slice |  308 ième => Python 

### Mask -> RTSTRUCT file 


In [ ]:
from library_dicom.rtss_processor.model.RTSS_Writer import RTSS_Writer
import pydicom

In [ ]:
dataset = {}
for i in range(1,11):
    data = {}
    data['ROIName'] = str(i)
    data['ROIGenerationAlgorithm'] = 'SEMIAUTOMATIC'
    dataset[i] = data
print(dataset)    

In [ ]:
new_rtss = RTSS_Writer(mask_4D, ct_path, dataset)

In [ ]:
filename = 'rtss_2.dcm'
directory_path = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/RT_STRUCT'
new_rtss.save_file(filename, directory_path)